# Patent Analysis

In [56]:
%%capture
# !pip install -r requirements.txt


## Importing Libraries

In [57]:
%%capture
import sys
import importlib
sys.path.append('src')


import preprocess_data
import qaagent
import koragent

importlib.reload(preprocess_data)
importlib.reload(qaagent)
importlib.reload(koragent)


In [58]:
import nltk
nltk.download('all', quiet=True)

True

## Downloading patents and preprocessing

In [59]:
year = 2023
month = 1
day = 5

In [60]:
saved_patent_names = preprocess_data.parse_and_save_patents(year, month, day, False)

File c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\ipa230105 already exists. Skipping download.
File c:\Users\armin\Documents\GitRepo\uspto_patent_analysis\data\ipa230105 already exists. Skipping extract.


## Loading data and querying sample file


In [61]:
prompt = """
Task: Carefully review the given patent text and extract as much physical measurements information such as length/distance, mass/weight, time, temperature, Volume, area, speed, pressure, energy, power, electric current 
and voltage, frequency, force, acceleration, density, resistivity, magnetic field strength, and luminous intensity as much as possible. 
We are particularly interested in physical measurements including substance that was measured, Value of the measurement, and Unit of the measurement, and measurement type mentioned in the text. 

For each measurement, please provide the following details:
- The substance that was measured. (substance)
- The specific value or range that was measured. (Measured Value)
- The unit of the measurement, if provided. (Unit)
- The type of measurement being conducted (e.g., diameter, size, etc.) 


Format your response in a structured JSON-like format, as follows:

{"Content": [
    {
      "Measurement_substance": "substance",
      "Measured_value": "value",
      "Measured_unit": "unit",
      "measurement_type": "type"
    },
    // ... additional measurements, if present
  ]
}

If multiple measurements are present in the text, each should be listed as a separate object within the "Content" array.

Example: If the text includes the sentence, "The resulting BaCO3 had a crystallite size of between about 20 and 40 nm", the output should be:

{"Content": [
    {
      "Measurement_substance": "BaCO3",
      "Measured_value": "between about 20 and 40",
      "Measured_unit": "nm",
      "measurement_type": "crystallite size"
    }
  ]
}

Try to provide as complete and accurate information as possible. Print only the formatted JSON response.
"""

In [62]:
saved_patent_names =  ['US20230001042A1-20230105.XML.txt',
                       'US20230001385A1-20230105.XML.txt',
                       'US20230002279A1-20230105.XML.txt',
                       'US20230002549A1-20230105.XML.txt',
                       'US20230002554A1-20230105.XML.txt',
                       'US20230002555A1-20230105.XML.txt',
                       'US20230002559A1-20230105.XML.txt'
                       ]

In [63]:
import json
model_name = "gpt-3.5-turbo"
gpt_3_results = {}
total_cost_gpt3 = 0
for i in range(len(saved_patent_names)):
    cost, output = qaagent.call_QA_to_json(prompt, year, month, day, saved_patent_names, i, False, model_name)
    total_cost_gpt3 += cost
    gpt_3_results[saved_patent_names[i]] = json.loads(output)


In [64]:
import json
model_name = "gpt-4"
gpt_4_results = {}
total_cost_gpt4 = 0
for i in range(len(saved_patent_names)):
    cost, output = qaagent.call_QA_to_json(prompt, year, month, day, saved_patent_names, i, False, model_name)
    total_cost_gpt4 += cost
    gpt_4_results[saved_patent_names[i]] = json.loads(output)


Results for 7 files:
Run Time: 
- GPT-3.5: 161.4 s
- GPT-4: 453.2 s

Cost:
- GPT-3.5: $0.023
- GPT-4: $0.54